In [1]:
import pyspark as pys
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf


#spark_session = SparkSession.builder.master("spark://192.168.1.8:7077").getOrCreate()

#sparkC = spark_session.sparkContext
sparkC = pys.SparkContext()

file_en = sparkC.textFile("/home/ubuntu/europarl-v7.sv-en.en")
file_en.cache()
file_sv = sparkC.textFile("/home/ubuntu/europarl-v7.sv-en.sv")
file_sv.cache()

def swap_key_values(tf):
        return tf.map(lambda tf: (tf[1], tf[0]))

def convert_case(tf):
        return tf.map(lambda x: x.lower())

# Convert To Lower Case
file_en = convert_case(file_en)
file_sv = convert_case(file_sv)

# Zipping
file_en = file_en.zipWithIndex()
file_sv = file_sv.zipWithIndex()

# Swap Key Values
file_en = swap_key_values(file_en)
file_sv = swap_key_values(file_sv)

# Merging
file_merged = file_en.join(file_sv)
file_merged = file_merged.sortBy(lambda a: a[0])

# Same No of Words
smow = file_merged.filter(lambda x: len(x[1][0].split())==len(x[1][1].split()))

lines_en = smow.map(lambda a: a[1][0])
lines_sv = smow.map(lambda a: a[1][1])

tokens_lines_en = lines_en.map(lambda a: a.split())
tokens_lines_sv = lines_sv.map(lambda a: a.split())

# Construct Token Tuples
tokenTuple = tokens_lines_en.zip(tokens_lines_sv)

# Construct Word Pair Tuples
wordpairTuple = tokenTuple.map(lambda a: list(zip(a[0],a[1])))

# Word Pair Fequency Flat Map
wordPairFrequencies = wordpairTuple.flatMap(lambda line: line).map(lambda word_pair: (word_pair, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda a: -a[1])

# 20 Values of Word Pair Frequencies From The Top
wordPairFrequencies.take(20)
wordPairFrequencies.saveAsTextFile("/home/ubuntu/result")
sparkC.stop()